# This file provides tool to extract target usrs from  a csv file efficiently. You can skip it.

In [88]:
from idCrawler import getUserProfile
from pymongo import MongoClient
from idCrawler import getTweets
import json
from urllib.request import urlopen

In [67]:
def patientsReader_bipolar(file_name,keywords=["borderline","bpd"]):
    f = open(file_name, 'r')
    patient_ids = {}
    valid_conditions = ['o','p','c']
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0].lower()
            description = segments[1].lower()
            screen_name = segments[2]
            user_id = segments[3]
            
        except:
            print(line)
            return patient_ids
        if condition in valid_conditions:
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
       
        
    return patient_ids

In [92]:
def patientsReader_BPD(file_name, keywords = ["bipolar"]):
    f = open(file_name, 'r')
    patient_ids = {}
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0]
            description = segments[1].lower()
            user_id = segments[2]
            
        except:
            print(line)
            return patient_ids
        if condition == 'o' or condition == 'O' or condition == 'p':
            if any(keyword in description for keyword in keywords):
                patient_ids[user_id] = 'c'
            else:
                patient_ids[user_id] = 'o'
            
        
    return patient_ids









def writeUsers(tweets, dbName, collectionName):
    collection = MongoClient("localhost", 27017)[dbName][collectionName]
    collection.insert(tweets)
    print("{} tweets inserted".format(len(tweets)))
    
def sentiment_analyize(tweets):
    payload = {"data": [],"language": "en"}
    
    for tweet in tweets:
        payload["data"].append({"text": tweet["text"],"id": tweet["id"]})
    
    payload = json.dumps(payload).encode('utf-8')
    response = urlopen('http://www.sentiment140.com/api/bulkClassifyJson?appid=ccha97u@gmail.com', payload) # request to server
    results = response.read().decode('"ISO-8859-1"') # get the response
    results = json.loads(results)['data']
    for i,result in enumerate(results):
       
        if result['polarity'] == 0:
            tweets[i]["polarity"] = "negative"
        elif result['polarity'] == 2:
            tweets[i]["polarity"] = "neutral"
        elif result['polarity'] == 4:
            tweets[i]["polarity"] = "positive"
        else:
             tweets[i]["polarity"] = "unknown"
    return tweets


def scan_tweets(tweets, bulk_size=10000):

    buffer = []
    sentiment_tweets = []
    for tweet in tweets:
        buffer.append(tweet)
        if len(buffer) >= bulk_size:
            print("10000 tweets  processed")
            sentiment_tweets += sentiment_analyize(buffer)
            buffer = []
    
    
    sentiment_tweets += sentiment_analyize(buffer)
    
    return sentiment_tweets

In [119]:
def expertFilter_BPD(file_name, keywords = ["bipolar"]):
    f = open(file_name, 'r')
    patient_ids = {}
    for line in f:
        try:
            segments = line.strip().split("\t")
            condition = segments[0].lower()
            description = segments[1].lower()
            user_id = segments[2]
            
        except:
            print(line)
            return patient_ids
        
        if condition ==  'x' and any(keyword in description for keyword in keywords):
            patient_ids[user_id] = description
            
            
        
    return patient_ids

file_location = "tagged_bpd_descriptions.tsv"
keywords = ["doctor","expert", "psychologist", "therapist", "coach", "advisor","certified","famous","specialist", "clinics","clinical","research"]
BPD_experts = expertFilter_BPD(file_location,keywords)
print(len(BPD_experts))
for user_id, description in BPD_experts.items():
    print(description + "\n")

22
blog from an, clinically cured, experience expert about borderline personality desorder and mentalisation based treatment

finding the right in between. positive,insightful &  humor about #recovery #bpd #ptsd #ed #addiction #dbt. i'm a certified recovery coach,peer advocate & writer

#bpd can't stop me or you! / martha beck life coach/heartmath certified coach/dbt skills trainer/family & relationship support/motivational speaker

i am an author, life bpd & mental health coach working with people with bpd, bpd loved ones & people with mental illness wanting to cope more effectively.

#lifecoach #marriage #coaching #mentalhealth #emotionmanagement using #dbt and #christian principles to handle difficult life situations. #bpd

hr/recruitment specialist;bpd/ocpd/add/chronic anxiety & depression;phd in chemistry,a recovering perfectionist

live,laugh,love xxx helped a loved one recover from bpd. fighting the stigma about mental illness. qualified hypnotherapist xxxx

psychotherapeut (hp)

In [118]:
def expertFilter_bipolar(file_names, keywords = ["bipolar"]):
    patient_ids = {}
    for file_name in file_names:
        f = open(file_name, 'r')

        for line in f:
            try:
                segments = line.strip().split("\t")
                condition = segments[0].lower()
                description = segments[1].lower()
                user_id = segments[3]

            except:
                break

            if condition ==  '' and any(keyword in description for keyword in keywords):
                patient_ids[user_id] = description

            
        
    return patient_ids

keywords = ["doctor","expert", "psychologist", "therapist", "coach", "advisor","certified","famous","specialist", "clinics","clinical","research"]

mix = []
file_locations = ["bipolar_profile_700_1400.csv", "bipolar_profile.tsv"]
experts_bipolar = expertFilter_bipolar(file_locations, keywords)
print(len(experts_bipolar))

    
w = open("bipolar_experts.tsv", "w")
for expert_id, description in experts_bipolar.items():
    w.write("\t{}\t{}\n".format(description, expert_id))
w.close()


67
